In [ ]:
from pylab import*
import numpy as np
import scipy.stats 
from scipy.special import gamma
import seaborn as sns
from scipy.stats import beta
from scipy.stats import spearmanr
import pandas as pd
from sklearn import linear_model

In [ ]:
#Vamos a ver la base, que consta de lote que una variable categorica, horas como medida de tiempo y monto en dinero
ruta = "./data/"
bd="lineal_horm.xlsx"
bd_horm=pd.read_excel(ruta+bd)
bd_horm

In [ ]:
#Vamos a llamar a la regresion lineal para usarla como reg cuando se necesite 
reg = linear_model.LinearRegression()

In [ ]:
#Vamos a usar a x como variable explicativa (horas), vamos a usar y como variable de respuesta (monto), queremos explicar el monto
#a traves del tiempo
X = bd_horm[["horas"]]
y = bd_horm[["monto"]]

In [ ]:
#Hacemos la regresion lineal de x con y
reg.fit(X, y)

In [ ]:
#obtenemos el beta_1 de la regresion
reg.coef_

In [ ]:
#obtenemos el beta_0 para la regresion
reg.intercept_

In [ ]:
#predecimos valores de la regresion que son las valores en la recta para cada punto en el tiempo x variable explicativa
Y_pred = reg.predict(X) 
Y_pred

In [ ]:
#Hacemos un cambio en la visualizacion de los valores predichos para que sea como una lista
y_estim=Y_pred.reshape(-1)
y_estim


In [ ]:
#Transformamos los datos de x para que sea mas facil de manejarlos
x=X.to_numpy().reshape(-1)
x

In [ ]:
#Volvemos a transformar a y para que sea mas facil de manejar, pero son los datos reales de la base de datos
y_real=y.to_numpy()
y_real=y_real.reshape(-1)
n=len(y_real)
y_real

In [ ]:
#obtenemos los residuales como la diferencia de los valores reales de la base con los predichos por el modelo
residual=y_real-y_estim
residual

In [ ]:
#obtenemos el error estandar de la correlacion bajo la muestra completa, sin muestras bootstrap
den=sum((x-np.mean(x))**2)
num=sum(residual**2)/(n)
err_reg1=np.sqrt(num/den)
err_reg1


In [ ]:
#Hacemos el plot de la regresion de horas vs monto, vemos que tiene pendiente negativa
fig, ax = plt.subplots(figsize=(9,6))
plt.scatter(X, y,s=30)
plt.plot(X, Y_pred, color='red')
xlabel("Horas",size=14)
ylabel("Monto",size=14)
xticks(fontsize=16)
yticks(fontsize=16)
show()

In [ ]:
#Nos tomamos un vector vacio para llenarlos con errores estandar de la regresion, que se toma con muestras de tamanio 5 y con
#remplazo, hasta que sean 200 iteraciones, por cada regresion nos tomamos los residuales y calculamos el error de la correlacion. La muestra 
#se toma de la base original tomando renglones completos del data frame agrupado por lote, una vez tenemos la muestra, nos tomamos 
#las horas y el monto para hacer el modelo de regresion
err_boost=np.array([])
for i in range(0,200): 
    bd_random=bd_horm.groupby("lote").sample(n=5,replace=True)
    X = bd_random[["horas"]]
    y = bd_random[["monto"]]
    reg.fit(X,y)
    Y_pred = reg.predict(X)
    y_estim=Y_pred.reshape(-1)
    x=X.to_numpy().reshape(-1)
    y_real=y.to_numpy()
    y_real=y_real.reshape(-1)
    n=len(y_real)
    residual=y_real-y_estim
    den=sum((x-np.mean(x))**2)
    num=sum(residual**2)/(n) 
    err_reg=np.sqrt(num/den)
    err_boost=np.append(err_boost,err_reg)

In [ ]:
#Vemos el vector de errores con base en cada muestra 
print(err_boost)

In [ ]:
#ploteamos el histograma de las correlaciones, con 
def hist_corr(cor_boost,corr1):
    fig, ax = plt.subplots(figsize=(9,6))
    hist(cor_boost,bins=50,alpha=0.5,edgecolor="white",label="Corr Boostrap")
    axvline(x=corr1,color='green', linestyle='--',linewidth=3,label="Correlacion sin boostrap")
    axvline(x=np.mean(cor_boost),color='black', linestyle='--',linewidth=3,label="correlación Boostrap$")

    axvline(percentile(cor_boost,2.5),color="blue",linestyle="--",linewidth=2)                                                                                                                                                    
    axvline(percentile(cor_boost,97.5),color="blue",linestyle="--",linewidth=2) 
    show()
    print("Correlacion sin Bootsrap:", corr1)
    print("Promedio de la Correlacion de las muestras Bootstrap:", np.mean(cor_boost))
    print("(", percentile(cor_boost,2.5),",",percentile(cor_boost,97.5),")")

In [ ]:
hist_corr(err_boost,err_reg1)